In [1]:
!pip install transformers accelerate sentencepiece

## 1. Import libraries

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import json
import torch
import time
from typing import List, Dict, Any

transformers → loads TinyLlama model + tokenizer + pipeline.

json → loads rubric and submissions from dataset files.

torch → checks for GPU (CUDA).

time → measures inference speed.

typing → type hints for functions.

 ## 2. Load TinyLlama on GPU

In [3]:
# Load TinyLlama on GPU
# -----------------------------
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU' if device==0 else 'CPU'}")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto" if device == 0 else None,  # Auto-allocate layers to GPU
    torch_dtype=torch.float16 if device == 0 else torch.float32  # Use half precision on GPU
)

chat_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device,               # GPU if available
    max_new_tokens=512,
    temperature=0.7
)

Using device: CPU


Device set to use cpu


Loads TinyLlama model.

Uses GPU if available (device=0), else CPU (device=-1).

Uses half precision (float16) on GPU → faster & lighter.

Wraps everything in a pipeline for easy text generation.

## 4. Academic Integrity Detection (AI/Human/Hybrid classifier)

In [4]:
# System prompt for strict JSON
# -----------------------------
SYSTEM_PROMPT = "You are a careful academic assistant. Be precise and return strict JSON."

# -----------------------------
# Academic Integrity Detection
# -----------------------------
def build_detection_prompt(submission: str, few_shots: List[Dict[str, Any]]) -> List[Dict[str, str]]:
    shot_texts = []
    for s in few_shots:
        shot_texts.append(
            f'Submission: """{s.get("final_submission","")}"""\n'
            f'Your analysis (2–4 bullet points): <analysis>\n'
            f'Label: {s.get("label_type","")}\n'
        )
    examples_block = "\n\n".join(shot_texts) if shot_texts else "/* no examples available */"

    user = f"""
You are an AI text-source classifier for academic integrity.
Decide whether the student submission is Human, AI, or Hybrid (AI-assisted).

Guidelines:
- Consider discourse features, style consistency, coherence, repetitiveness, and cliché patterns.
- Hybrid = meaningful human writing with some AI assistance.

Examples:
{examples_block}

Analyze the NEW submission and return STRICT JSON.
NEW submission:
\"\"\"{submission}\"\"\"\n
Answer only with JSON.
"""
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

Builds a classification prompt:

Model classifies submission into Human / AI / Hybrid.

Few-shot examples help it learn how to classify.

## 5. Rubric-Aligned Feedback

In [5]:
# Rubric-Aligned Feedback
# -----------------------------
def build_feedback_prompt(domain: str, assignment_prompt: str, rubric_text: str, submission: str) -> List[Dict[str, str]]:
    user = f"""
You are a supportive assessor. Provide actionable feedback aligned to the rubric.
Return a STRUCTURED report (no extraneous text).

Sections:
1) "overall_summary": 2–4 sentences on strengths and priorities.
2) "criteria_feedback": array of items per rubric criterion with:
   - "criterion_id"
   - "rating": ["excellent","good","average","needs_improvement","poor"]
   - "evidence": 1–3 bullet points citing examples
   - "improvement_tip": one concrete step

Context:
- Domain: {domain}
- Assignment prompt: {assignment_prompt}

Rubric:
{rubric_text}

Submission:
\"\"\"{submission}\"\"\"\n

Constraints:
- Be concise but specific. Output MUST be valid JSON with keys: overall_summary, criteria_feedback, suggested_grade.
"""
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]


Builds a feedback prompt:

Summarizes strengths & weaknesses.

Aligns with rubric criteria.

Outputs structured JSON: overall_summary, criteria_feedback, suggested_grade.

## 6. Rubric formatting helper

In [6]:
# Rubric formatting helper
# -----------------------------
def format_rubric(rubric):
    formatted_rubric = f"Rubric ID: {rubric['rubric_id']}\n\nCriteria:\n"
    for item in rubric['criteria']:
        formatted_rubric += f"Criterion: {item['criterion_id']}\nName: {item['name']}\nDescription: {item['description']}\nPerformance Descriptors:\n"
        for key, val in item['performance_descriptors'].items():
            formatted_rubric += f"  - {key}: {val}\n"
    return formatted_rubric



## 7. Psychology Load data / Build feedback prompt / Generate feedback

In [7]:
# Load data Psychology Training Data
# -----------------------------
with open("psychology.json") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
submission_text = data['submissions'][0]['final_submission']

# -----------------------------
# Build feedback prompt
# -----------------------------
feedback_prompt = build_feedback_prompt(
    domain="psychology",
    assignment_prompt="Analyze student submission and provide detailed feedback",
    rubric_text=rubric_text,
    submission=submission_text
)

# Combine roles into single prompt
full_prompt = ""
for role_item in feedback_prompt:
    full_prompt += f"[{role_item['role'].upper()}]: {role_item['content']}\n"

# -----------------------------
# Generate feedback
# -----------------------------
t1 = time.time()
response = chat_pipe(full_prompt)[0]['generated_text']
t2 = time.time()

print(f"Time taken: {t2 - t1:.2f} seconds\n")
print(response)

Time taken: 71.69 seconds

[SYSTEM]: You are a careful academic assistant. Be precise and return strict JSON.
[USER]: 
You are a supportive assessor. Provide actionable feedback aligned to the rubric.
Return a STRUCTURED report (no extraneous text).

Sections:
1) "overall_summary": 2–4 sentences on strengths and priorities.
2) "criteria_feedback": array of items per rubric criterion with:
   - "criterion_id"
   - "rating": ["excellent","good","average","needs_improvement","poor"]
   - "evidence": 1–3 bullet points citing examples
   - "improvement_tip": one concrete step

Context:
- Domain: psychology
- Assignment prompt: Analyze student submission and provide detailed feedback

Rubric:
Rubric ID: rub_psychology_0001

Criteria:
Criterion: c1
Name: Conceptual Understanding of Biases
Description: Demonstrates comprehension of selected biases, including theoretical foundations.
Performance Descriptors:
  - excellent: Shows precise and in-depth understanding of cognitive biases, with clear

## 8. Engineering Load data / Build feedback prompt / Generate feedback

In [8]:
# Load data Teaching Training Data
# -----------------------------
with open("engineering.json") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
submission_text = data['submissions'][0]['final_submission']

# -----------------------------
# Build feedback prompt
# -----------------------------
feedback_prompt = build_feedback_prompt(
    domain="engineering",
    assignment_prompt="Analyze student submission and provide detailed feedback",
    rubric_text=rubric_text,
    submission=submission_text
)

# Combine roles into single prompt
full_prompt = ""
for role_item in feedback_prompt:
    full_prompt += f"[{role_item['role'].upper()}]: {role_item['content']}\n"

# -----------------------------
# Generate feedback
# -----------------------------
t1 = time.time()
response = chat_pipe(full_prompt)[0]['generated_text']
t2 = time.time()

print(f"Time taken: {t2 - t1:.2f} seconds\n")
print(response)


Time taken: 484.17 seconds

[SYSTEM]: You are a careful academic assistant. Be precise and return strict JSON.
[USER]: 
You are a supportive assessor. Provide actionable feedback aligned to the rubric.
Return a STRUCTURED report (no extraneous text).

Sections:
1) "overall_summary": 2–4 sentences on strengths and priorities.
2) "criteria_feedback": array of items per rubric criterion with:
   - "criterion_id"
   - "rating": ["excellent","good","average","needs_improvement","poor"]
   - "evidence": 1–3 bullet points citing examples
   - "improvement_tip": one concrete step

Context:
- Domain: engineering
- Assignment prompt: Analyze student submission and provide detailed feedback

Rubric:
Rubric ID: rub_engineering_0011

Criteria:
Criterion: c1
Name: Scope & Comprehensiveness
Description: Explains the entire process from initial design to final assembly, including all key phases.
Performance Descriptors:
  - excellent: Provides an exceptionally thorough and detailed account of all phas

## 9. Engineering Load data / Build feedback prompt / Generate feedback

In [9]:
# Load data IT Training Data
# -----------------------------
with open("it.json") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
submission_text = data['submissions'][0]['final_submission']

# -----------------------------
# Build feedback prompt
# -----------------------------
feedback_prompt = build_feedback_prompt(
    domain="IT",
    assignment_prompt="Analyze student submission and provide detailed feedback",
    rubric_text=rubric_text,
    submission=submission_text
)

# Combine roles into single prompt
full_prompt = ""
for role_item in feedback_prompt:
    full_prompt += f"[{role_item['role'].upper()}]: {role_item['content']}\n"

# -----------------------------
# Generate feedback
# -----------------------------
t1 = time.time()
response = chat_pipe(full_prompt)[0]['generated_text']
t2 = time.time()

print(f"Time taken: {t2 - t1:.2f} seconds\n")
print(response)


Time taken: 71.02 seconds

[SYSTEM]: You are a careful academic assistant. Be precise and return strict JSON.
[USER]: 
You are a supportive assessor. Provide actionable feedback aligned to the rubric.
Return a STRUCTURED report (no extraneous text).

Sections:
1) "overall_summary": 2–4 sentences on strengths and priorities.
2) "criteria_feedback": array of items per rubric criterion with:
   - "criterion_id"
   - "rating": ["excellent","good","average","needs_improvement","poor"]
   - "evidence": 1–3 bullet points citing examples
   - "improvement_tip": one concrete step

Context:
- Domain: IT
- Assignment prompt: Analyze student submission and provide detailed feedback

Rubric:
Rubric ID: rub_it_0002

Criteria:
Criterion: c1
Name: Understanding of Topic
Description: Demonstrates clear and accurate understanding of AI and its impact on cybersecurity.
Performance Descriptors:
  - excellent: Displays comprehensive knowledge and insightful connections to the topic.
  - good: Shows clear u

## 10. Engineering Load data / Build feedback prompt / Generate feedback

In [10]:
# Load data Teaching Training Data
# -----------------------------
with open("teaching.json") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
submission_text = data['submissions'][0]['final_submission']

# -----------------------------
# Build feedback prompt
# -----------------------------
feedback_prompt = build_feedback_prompt(
    domain="Teaching",
    assignment_prompt="Analyze student submission and provide detailed feedback",
    rubric_text=rubric_text,
    submission=submission_text
)

# Combine roles into single prompt
full_prompt = ""
for role_item in feedback_prompt:
    full_prompt += f"[{role_item['role'].upper()}]: {role_item['content']}\n"

# -----------------------------
# Generate feedback
# -----------------------------
t1 = time.time()
response = chat_pipe(full_prompt)[0]['generated_text']
t2 = time.time()

print(f"Time taken: {t2 - t1:.2f} seconds\n")
print(response)


Time taken: 479.28 seconds

[SYSTEM]: You are a careful academic assistant. Be precise and return strict JSON.
[USER]: 
You are a supportive assessor. Provide actionable feedback aligned to the rubric.
Return a STRUCTURED report (no extraneous text).

Sections:
1) "overall_summary": 2–4 sentences on strengths and priorities.
2) "criteria_feedback": array of items per rubric criterion with:
   - "criterion_id"
   - "rating": ["excellent","good","average","needs_improvement","poor"]
   - "evidence": 1–3 bullet points citing examples
   - "improvement_tip": one concrete step

Context:
- Domain: Teaching
- Assignment prompt: Analyze student submission and provide detailed feedback

Rubric:
Rubric ID: rub_teaching_0025

Criteria:
Criterion: c1
Name: Understanding of Early Literacy Development
Description: Demonstrates knowledge of foundational early literacy skills (e.g., phonological awareness, vocabulary, print knowledge) and developmental milestones for ages 3 to 6 years.
Performance Des

## 11. Engineering Load data / Build feedback prompt / Generate feedback

In [11]:
# Load data
# -----------------------------
# -----------------------------
with open("accounting.json", "r", encoding="utf-8") as f:
    data = json.load(f)

rubric_text = format_rubric(data['rubric'])
submission_text = data['submissions'][0]['final_submission']

# -----------------------------
# Build feedback prompt
# -----------------------------
feedback_prompt = build_feedback_prompt(
    domain="Accounting",
    assignment_prompt="Analyze student submission and provide detailed feedback",
    rubric_text=rubric_text,
    submission=submission_text
)

# Combine roles into single prompt
full_prompt = ""
for role_item in feedback_prompt:
    full_prompt += f"[{role_item['role'].upper()}]: {role_item['content']}\n"

# -----------------------------
# Generate feedback
# -----------------------------
t1 = time.time()
response = chat_pipe(full_prompt)[0]['generated_text']
t2 = time.time()

print(f"Time taken: {t2 - t1:.2f} seconds\n")
print(response)


Time taken: 411.20 seconds

[SYSTEM]: You are a careful academic assistant. Be precise and return strict JSON.
[USER]: 
You are a supportive assessor. Provide actionable feedback aligned to the rubric.
Return a STRUCTURED report (no extraneous text).

Sections:
1) "overall_summary": 2–4 sentences on strengths and priorities.
2) "criteria_feedback": array of items per rubric criterion with:
   - "criterion_id"
   - "rating": ["excellent","good","average","needs_improvement","poor"]
   - "evidence": 1–3 bullet points citing examples
   - "improvement_tip": one concrete step

Context:
- Domain: Accounting
- Assignment prompt: Analyze student submission and provide detailed feedback

Rubric:
Rubric ID: rub_accounting_0001

Criteria:
Criterion: c1
Name: Understanding of Blockchain Concepts
Description: Demonstrates knowledge of blockchain principles (decentralization, immutability, smart contracts) and their relevance to accounting.
Performance Descriptors:
  - excellent: Clearly explains m